# Courseroad Optimization

## Installing Packages

In [ ]:
# Install packages
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("Plots")
Pkg.add("JSON")
Pkg.add("PyCall")

In [1]:
using CSV, DataFrames, JuMP, Gurobi, Plots, PyCall, DelimitedFiles

In [2]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/bella/.julia/conda/3/lib/python3.10/site-packages/numpy/__init__.py'>

In [3]:
ENV["GRB_LICENSE_FILE"] = "/Users/bella/Downloads/6.7201/gurobi.lic"

"/Users/bella/Downloads/6.7201/gurobi.lic"

## Importing and Setting up Data

In [4]:
data_6_4  = CSV.read("6-4_requirements.csv", DataFrame)
course_data = CSV.read("course_data.csv", DataFrame)
data = innerjoin(course_data, data_6_4, on = :no)
sort!(data, :no)
typeof(data)

DataFrame

In [ ]:
# my attempt at loading in pre_reqs.txt but this runs into issues between np and julia
# loaded_arr = np.loadtxt("pre_reqs.txt") 
# PR = np.reshape(loaded_arr, (size(loaded_arr, 0), size(loaded_arr, 1) // arr.shape[2], arr.shape[2]))
loaded_arr = readdlm("pre_reqs.txt")

# Calculate the dimensions for reshaping
m = size(loaded_arr, 1)  # Size of the first dimension
n = size(loaded_arr, 2) ÷ m  # Size of the second dimension divided by m
o = m  # Set p equal to m

# Reshape the array
P = reshape(loaded_arr, m, o, n)

In [5]:
# Build sets
classes = 1:nrow(data)
semesters = 1:8
typeof(data)

DataFrame

### Data Directly from CSV

In [6]:
# Get basic class information
names = data."no"
fall = data."fa"
spring = data."sp"
U = data."vu" # units
R = data."ra" # ratings
H = data."h" # hours
same_as = data."sa"
pre_reqs = data."pr"


# Get GIR-related information
hh = data."hh"
ha = data."ha"
hs = data."hs"
he = data."he"
cih = data."ci"
ci_hw = data."cw"
rest = data."re"
lab = data."la"
bio = data."bio"
chem = data."ch"
phys1 = data."p1"
phys2 = data."p2"
calc1 = data."c1"
calc2 = data."c2"

# Get 6-4-related information
programming = data."p-64"
foundation = data."f-64"
math2 = data."m2-64"
math3 = data."m3-64"
datac = data."dac-64"
mod = data."moc-64"
decision = data."dec-64"
compute = data."coc-64"
human = data."huc-64"
cim_6_4 = data."cim-64"
cim2 = data."cim2-64"
serc = data."serc"
aus = data."aus"
eecs = data."eecs"

3535-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

### Data Processing

In [33]:
# Determining which classes are hasses
hass = [(hh[i] == 1 || ha[i] == 1 || hs[i] == 1 || he[i] == 1) ? 1 : 0 for i in classes]

# Making matrix of GIR requirements 
G = hcat(ha, hs, hh, cih, hass, bio, phys1, phys2, calc1, calc2, chem, rest, lab)
G_sat = [1, 1, 1, 2, 8, 1, 1, 1, 1, 1, 1, 12, 12]

# Making matrix for fall/spring classes
A = hcat(fall, spring)

# Make vector indicating whether a semester is a fall (1) or spring (2) semester
S = [ (j%2 == 1) ? 1 : 2 for j in semesters]

# Get 1801A and 1802A indices
calc1A = findfirst(item -> item == "18.01A", names)
calc2A = findfirst(item -> item == "18.02A", names)

# Making matrices for 6-4 requirements. 
center = [ (datac[i] == 1 || mod[i] == 1 || decision[i] == 1 || compute[i] == 1 || human[i] == 1) ? 1 : 0 for i in classes]
aus_cim6_4 = [ (aus[i] == 1 || cim_6_4[i] == 1) ? 1 : 0 for i in classes]
aus_cim2 = [ (aus[i] == 1 || cim2[i] == 1) ? 1 : 0 for i in classes]

M = hcat(programming, foundation, math2, math3, center, datac, mod, decision, compute, human, serc, cim_6_4, cim2, aus_cim6_4, aus_cim2)
M_sat = [1, 3, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3]        
EM = [ (eecs[i] == 1 || names[i][1:2] == "18") ? 1 : 0 for i in classes]

# Making matrix for pre-reqs
max_prs = 1:8

1:8

## Tweakable variables

In [28]:
a = 0.5

0.0

## Building the Model

### Index references
i: class

j: semester

k: index of GIR requirement

l: index of 6-4 requirement

### Model Definition

In [50]:
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, X[classes, semesters], Bin);

# Objective
@objective(model, Min, a*sum(H[i]*sum(X[i,j] for j in semesters) for i in classes) + (1- a)*sum(R[i]*sum(X[i,j] for j in semesters) for i in classes));

print("objective")
# ---- General Constraints ------

# Cannot take a class more than once
@constraint(model, num_times[i in classes], sum(X[i,j] for j in semesters) <= 1);

# Maximum and minimum number of units per semester
@constraint(model, max_units[j in semesters], sum(U[i]*X[i,j] for i in classes) <= 60);
@constraint(model, min_units[j in semesters], sum(U[i]*X[i,j] for i in classes) >= 36);

# Fall classes taken in fall, spring classes taken in spring
@constraint(model, fall_spring[i in classes, j in semesters], X[i,j] - A[i, S[j]] <= 0);

# Pre-reqs
# @constraint(model, pre_req_constraint[i in classes, j in semesters, k in max_prs], X[i,j]*sum(sum(X[z,w] for w in j:8)*P[i, z, k] for z in classes) >= X[i,j]*1)
print("general")
# ---- GIR Constraints ------

# # GIR requirements by classes
@constraint(model, gir_classes[k in 1:11], sum(sum(G[i,k]X[i,j] for j in semesters) for i in classes) >= G_sat[k]);

# # GIR requirements by units
@constraint(model, gir_units[k in 12:13], sum(sum(G[i,k]X[i,j]U[i] for j in semesters) for i in classes) >= G_sat[k]);

print("2 left")
# # at least 180 units beyond GIR
@constraint(model, units_beyond_GIR, sum(
        sum(U[i]*X[i,j] for j in semesters)
        - sum(U[i]*X[i,j] for j in semesters)
        * (sum(G[i,k] for k in 1:13) >= 1 ? 1 : 0) for i in classes) >= 180);

# Takes 1801A and 1802A consecutively
@constraint(model, calcA_sameas[j in 1:7], X[calc1A,j] == X[calc2A,j])


# # ---- 6-4 Constraints ------

# General requirements satisfied
@constraint(model, general_64[l in 3:15], sum(M[i, l]*sum(X[i,j] for j in semesters) for i in classes) >= M_sat[l])

# One additional math/eecs course
@constraint(model, math_eecs[l in 3:15], sum(EM[i] * sum(X[i,j] for j in semesters) * M[i,l] >= 1 ? ((sum(M[b,l] * sum(X[b,j] for j in semesters) for b in classes)) <= 0 ? 0 : 1) : 1 for i in classes) >= 1)



Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
objectivegeneral2 left

LoadError: MethodError: no method matching isless(::Int64, ::AffExpr)
[0mClosest candidates are:
[0m  isless(::Integer, [91m::ForwardDiff.Dual{Ty}[39m) where Ty at ~/.julia/packages/ForwardDiff/QdStj/src/dual.jl:145
[0m  isless(::Real, [91m::AbstractFloat[39m) at operators.jl:185
[0m  isless(::Real, [91m::ForwardDiff.Dual{Ty}[39m) where Ty at ~/.julia/packages/ForwardDiff/QdStj/src/dual.jl:145
[0m  ...

In [42]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31866 rows, 28280 columns and 160670 nonzeros
Model fingerprint: 0x29701b00
Variable types: 0 continuous, 28280 integer (28280 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 28918 rows and 14577 columns
Presolve time: 0.11s
Presolved: 2948 rows, 13703 columns, 72170 nonzeros
Variable types: 0 continuous, 13703 integer (13703 binary)

Root relaxation: infeasible, 116 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (116 simplex iterations

In [40]:
objective_value(model)

60.870000000000005

In [32]:
X_vals = convert(Matrix, value.(X))

for j in semesters
    classes_taken = []
    for i in classes
        if X_vals[i,j] == 1
            push!(classes_taken, names[i])
        end
    end
    println("semester", j, ": ", classes_taken)
end

semester1: Any[String7("6.3900"), String7("6.4110"), String7("6.7201"), String7("6.8611"), String7("6.S084")]
semester2: Any[String7("11.045"), String7("15.076"), String7("18.05"), String7("5.111"), String7("6.3720")]
semester3: Any[String7("18.01A"), String7("18.02A"), String7("21L.040"), String7("6.4120"), String7("6.8701")]
semester4: Any[String7("6.8301"), String7("7.014"), String7("HST.034")]
semester5: Any[String7("10.S28"), String7("8.02"), String7("HST.030"), String7("STS.053")]
semester6: Any[String7("HST.035"), String7("HST.207")]
semester7: Any[String7("21M.150"), String7("4.685"), String7("HST.031")]
semester8: Any[String7("21A.502"), String7("21L.002"), String7("21M.470"), String7("21M.475"), String7("8.011")]
